# Data cleaning
### This is a sample script for the data cleaning process in our project. The actual scripts would change depending on each dataset's characteristics and different purposes/models of using each dataset.

Team Member: Zeyu Gu, Yansong Bai, Yuzheng Zhang

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import lightgbm as lgb
import sklearn.metrics
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import confusion_matrix
from scipy import stats
import optuna
import warnings

In [ ]:
# Data preview
# data = pd.read_csv('city_ranking.csv')
data.describe()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
# Check the missing data
percent_missing = (data.isnull().sum() / len(data)).to_frame()
percent_missing.columns= ['Missing data %']
percent_missing.sort_values(by = 'Missing data %', ascending=False).head()

In [ ]:
# Visualise missing values for a sample of 500
msno.matrix(data.sample(500))

In [ ]:
# Drop the columns with percents of missing value exceed the threshold
# The threshold dependends on the specific attribute in the datasets and some special features
# should not be drop although the missing percent is high due to the 
drop_cols = [a for a in percent_missing[percent_missing["Missing data %"]>0.75].index]
data.drop(drop_cols,axis = 1,inplace= True)
data.head()

### Sometimes we just kept the missing value as NA since ML models like Graient Boosting Tree still have a good performance in tolerance of the existence of NA values.
### Othewise we used imputations for the missing value such as mean value imputation.

In [ ]:
# Sometimes we could just keep the missing value as NA 
# since some ML models like Graient Boosting Tree still have a good performancein tolerance of the existence of NA values
# Othewise we used imputation for the missing value ex: mean value imputation

data[data.columns[idx]] = data[data.columns[idx]].fillna(data[data.columns[idx]].meam()[0])

In [ ]:
# Estimate skewness and kurtosis
data.skew(), data.kurt()

In [ ]:
# Aggregate categories with fewer than 10 obs to the new category "Rare"
def transfer_rare_val(data,attr_name):
    val_counts = data[attr_name].value_counts()
    rare_val = val_counts.index[np.where(val_counts<=10)]
    data.loc[(data[attr_name].isin(rare_val)) & (data[attr_name]!=None),[attr_name]] = 'Rare'
    
    
transfer_rare_val(data,data.columns[2])
transfer_rare_val(data,data.columns[3])

In [ ]:
# Check outliers for columns that are normal distributed
def clean_outliers(idx):
    display(sns.boxplot(data=data[data.columns[idx]]))
    data.loc[(data[data.columns[idx]]<data[data.columns[idx]].mean() - 2 * data.std()[idx])|
              (data[data.columns[idx]]>data[data.columns[idx]].mean() + 2 * data.std()[idx]),
              data.columns[idx]]=np.nan

In [ ]:
clean_outliers(2)
clean_outliers(3)

In [ ]:
# Example when clean some special columns of data:

# Clean for ZIP
data.loc[(data['ZIP'].str.slice(0,2) != '89'),['ZIP']] = 'WrongValue'
data['ZIP'] = data['ZIP'].str.slice(0,5)
data[data['ZIP'].str.slice(0,2) != '89']

In [ ]:
# Clean for column of CITY_TYPE

# Set CITY_TYPE that are not 'Global','Cosmopolitan','Planned', 'Emergent' to None
data['CITY_TYPE'].value_counts()
data.loc[(data['CITY_TYPE'].isin(['Global','Cosmopolitan','Planned', 'Emergent'])),['CITY_TYPE']] = np.nan

In [ ]:
# Clean for column of LAT_LONG_RAW
data['LAT_LONG_RAW'] = data['LAT_LONG_RAW'].str.replace('(', '')
data['LAT_LONG_RAW'] = data['LAT_LONG_RAW'].str.replace(')', '')
data[['LAT','LONG']] = data['LAT_LONG_RAW'].str.split(',',expand=True)
data['LAT'] = abs(pd.to_numeric(data['LAT']))
data['LONG'] = abs(pd.to_numeric(data['LONG']))

### Encode features and detect multicollinearity as preparation before modeling (in our recommendation page):

In [ ]:
# Examine numerical features in the dataset
numeric_features = data.select_dtypes(include=[np.number])
numeric_features.columns

In [ ]:
data.select_dtypes(include='number').nunique()

In [ ]:
# Examine categorical features in the dataset
categorical_features = data.select_dtypes(include=[np.object])
categorical_features.columns

In [ ]:
# We use Label enoder to encode variables as the preparation for the ML models in our recommendation page
# example 1 
data[data.select_dtypes(include=['category']).columns] = data[data.select_dtypes(include=['category']).columns].astype(str)
data[categorical_features.columns] = data[categorical_features.columns].astype(str)
data[categorical_features.columns] = data[categorical_features.columns].apply(LabelEncoder().fit_transform)

# example 2 
categorical = data.columns[[4, 6, 8,10]]
ordinal = data.columns[[1,3,4,5]]
label = data.columns[[2,7]]

data[categorical] = data[categorical].astype('str')
enc = OrdinalEncoder()
enc.fit(data[ordinal])
data[ordinal] = enc.transform(data[ordinal])
data[label] = data[label].apply(LabelEncoder().fit_transform)
data[categorical] = data[categorical].astype('category')

In [ ]:
# Check the correlation between numerical variables
data[numeric_features.columns].corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Sometimes we need to check the existence of multicollinearity of potential independent variables
# So we get the VIF for each variable to detect multicollinearity

def get_vif(X_vif):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_vif.columns

    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i)
                              for i in range(len(X_vif.columns))]
    vif_data = vif_data.sort_values('VIF', ascending=False)
    print(vif_data)


In [ ]:
# Remove each variable with high one by one to avoid unnecessary removement and infomation loss
X_vif = data.drop("University Score", axis = 1)
get_vif(X_vif)

In [ ]:
X_vif = data.drop("Employment Score", axis = 1)
get_vif(X_vif)

In [ ]:
# Exploration graphs
# example:
fig, ax = plt.subplots()
ax.scatter(x = data[data.columns[idx]], y = y)
plt.ylabel('Satisfaction')
plt.xlabel('Food Ranking')
plt.show()

In [ ]:
# histplot example:
sns.histplot(data=data,x=data.columns[idx], hue='Preference')